In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing  and installing packages

In [ ]:
!pip install keras-tuner
import keras_tuner as kt

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import matplotlib.patches as patches
from pathlib import Path
import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from skimage import exposure
import tensorflow as tf
from PIL import Image
from progressbar import ProgressBar
from google.colab.patches import cv2_imshow
import os
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from keras.utils.vis_utils import plot_model

In [ ]:
tqdm.pandas(desc="my bar!")

# Preprocessing

In [ ]:
train_data=pd.read_csv("/content/drive/MyDrive/Pawpularity/train.csv")

In [ ]:
#creating a function to read an image and output its width, height and size in bytes
def image_size(path): 
    img=Image.open(path)
    return pd.Series([img.size[0], img.size[1], os.path.getsize(path)])

In [ ]:
#Getting the width, height and size in bytes of all images
paths="/content/drive/MyDrive/Pawpularity/train/"+train_data['Id']
pbar = ProgressBar()
image_sizes=[]
for p in pbar(paths):
  image_sizes.append(image_size(p))

In [ ]:
#Creating a dataframe with the width, height and size in bytes of all images
Images=pd.DataFrame(image_sizes)
Images=Images.rename(columns={0: "Width", 1: "Height", 2: "Size"}, errors="raise")
Images["Id"]=train_data["Id"]
Images.head()
Images.to_csv("/content/drive/MyDrive/Pawpularity/Imagesizes.csv")

In [ ]:
train_data['Id']=train_data['Id'] +".jpg"

In [ ]:
train_data,test_data=train_test_split(train_data,test_size=0.1,random_state=32)
train, valid = train_test_split(train_data, test_size=0.1)

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.5,
                                   rotation_range=45,
                                   width_shift_range=.15,
                                   height_shift_range=.15,
                                   horizontal_flip = True)
datagen1 = ImageDataGenerator(rescale = 1./255)

In [ ]:
Metacolumns = [
    'Subject Focus',
    'Eyes',
    'Face',
    'Near',
    'Action',
    'Accessory',
    'Group',
    'Collage',
    'Human',
    'Occlusion',
    'Info',
    'Blur',
    'Width', 
    'Height', 
    'Size'
]

# Convolution Neural Network

In [ ]:
# Creating a Cnn Model with input image size of (128,128,3)
target=(128,128)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu'),
  tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1,activation="linear")
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_squared_error'])

In [ ]:
#Generating batches of Images
x_train=datagen.flow_from_dataframe(dataframe=train,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col="Pawpularity",
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode="raw",
                                              batch_size=100)
x_valid=datagen1.flow_from_dataframe(dataframe=valid,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col="Pawpularity",
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode="raw",
                                              batch_size=100)
x_test=datagen1.flow_from_dataframe(dataframe=test_data,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col=None,
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode=None,
                                              batch_size=100)

Normal Fitting

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/checkpointCnn'
model_checkpoint_callback = [tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=2, mode='min', min_delta=0.01)]

In [ ]:
model.fit(x_train,epochs=10,validation_data=x_valid,callbacks=[model_checkpoint_callback])

In [ ]:
y_test=test_data["Pawpularity"]
y_test_pred=model.predict(x_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))

 K fold Cross Validation 

In [ ]:
#Creating a K fold instance which splits the training set into 5 splits 
from sklearn.model_selection import KFold
K = 5
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/checkpointCNN'
model_checkpoint_callback = [tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=2, mode='min', min_delta=0.01)]
result={}
j=1
for train_index, valid_index in kf.split(train_data):
  # Create data for this fold
  print( "\nFold ", j)
  x_train=datagen.flow_from_dataframe(dataframe=train_data.iloc[train_index,:],
                                            directory="/content/drive/MyDrive/Pawpularity/train",
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            weight_col=None,
                                            color_mode="rgb",
                                            target_size=target,
                                            class_mode="raw",
                                            batch_size=100)
  x_valid=datagen1.flow_from_dataframe(dataframe=train_data.iloc[valid_index,:],
                                            directory="/content/drive/MyDrive/Pawpularity/train",
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            weight_col=None,
                                            color_mode="rgb",
                                            class_mode="raw",
                                            target_size=target,
                                            batch_size=100)
  model.fit(x_train,epochs=5,validation_data=x_valid,callbacks=[model_checkpoint_callback])
  # Accumulate test set predictions
  j+=1
  result[j]=model.predict(x_test)

In [ ]:
y_test=test_data["Pawpularity"]
y_test_pred=model.predict(x_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))

# Transfer learning 

In [ ]:
# Loading the InceptionResNetV2 Model and specfing that only the last 10 layers of this model are trainable for our dataset 
target=(299,299)
base_model = tf.keras.applications.InceptionResNetV2(include_top=False,weights='imagenet',input_shape=(299,299,3))
for layer in base_model.layers[:-10]:
    layer.trainable = False

In [ ]:
# Loading the NASNetLarge Model and specfing that only the last 10 layers of this model are trainable for our dataset
target=(331,331)
base_model = tf.keras.applications.NASNetLarge(include_top=False,weights='imagenet')
for layer in base_model.layers[:-10]:
    layer.trainable = False

In [ ]:
# Loading the EfficientNetB1 Model and specfing that only the last 10 layers of this model are trainable for our dataset
target=(128,128)
base_model=tf.keras.applications.EfficientNetB1(include_top=True,weights="imagenet")
for layer in base_model.layers[:-10]:
    layer.trainable = False

In [ ]:
x_train=datagen.flow_from_dataframe(dataframe=train,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col="Pawpularity",
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode="raw",
                                              batch_size=100)
x_valid=datagen1.flow_from_dataframe(dataframe=valid,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col="Pawpularity",
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode="raw",
                                              batch_size=100)
x_test=datagen1.flow_from_dataframe(dataframe=test_data,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col=None,
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode=None,
                                              batch_size=100)

In [ ]:
# Creating a new model with the pretrained model plus some extra new layers
# This model works with the InceptionResNetV2 and NASNetLarge Model as the pretrained models 
model = tf.keras.models.Sequential([base_model,tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1,activation="linear")])

In [ ]:
# Creating a new model with the pretrained model plus some extra new layers
# This model works with the EfficientNetB1 as the pretrained model 
model = tf.keras.models.Sequential([base_model,
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1,activation="linear")])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_squared_error'])

Normal Fitting

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/checkpointEfficientNetB1'
model_checkpoint_callback = [tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=2, mode='min', min_delta=0.01)]

In [ ]:
model.fit(x_train,epochs=10,validation_data=x_valid,callbacks=[model_checkpoint_callback])

In [ ]:
y_test=test_data["Pawpularity"]
y_test_pred=model.predict(x_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))

 K fold Cross Validation 

In [ ]:
#Creating a K fold instance which splits the training set into 5 splits 
from sklearn.model_selection import KFold
K = 5
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/checkpointEfficientNetB1"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True)
result={}
j=1
for train_index, valid_index in kf.split(train_data):
  # Create data for this fold
  print( "\nFold ", j)
  x_train=datagen.flow_from_dataframe(dataframe=train_data.iloc[train_index,:],
                                            directory="/content/drive/MyDrive/Pawpularity/train",
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            weight_col=None,
                                            color_mode="rgb",
                                            target_size=target,
                                            class_mode="raw",
                                            batch_size=100)
  x_valid=datagen1.flow_from_dataframe(dataframe=train_data.iloc[valid_index,:],
                                            directory="/content/drive/MyDrive/Pawpularity/train",
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            weight_col=None,
                                            color_mode="rgb",
                                            class_mode="raw",
                                            target_size=target,
                                            batch_size=100)
  model.fit(x_train,epochs=5,validation_data=x_valid,callbacks=[model_checkpoint_callback])
  # Accumulate test set predictions
  j+=1
  result[j]=model.predict(x_test)

In [ ]:
y_test=test_data["Pawpularity"]
y_test_pred=model.predict(x_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))

# Meta + Image

In [ ]:
Meta=pd.read_csv("/content/drive/MyDrive/Pawpularity/Imagesizes.csv")
Meta['Id']=Meta['Id'] +".jpg"
Meta=Meta[['Id','Width', 'Height', 'Size']]

In [ ]:
train_data=pd.merge(train_data,Meta,on=['Id'],how="inner")
test_data=pd.merge(test_data,Meta,on=['Id'],how="inner")

In [ ]:
scaler = StandardScaler()
train_data[["Width","Height","Size"]] = scaler.fit_transform(train_data[["Width","Height","Size"]])
test_data[["Width","Height","Size"]] = scaler.transform(test_data[["Width","Height","Size"]])

In [ ]:
k = KMeans(8, random_state=0)
train_data['cluster'] = k.fit_predict(train_data.drop(['Id', 'Pawpularity'], axis=1))
test_data['cluster'] = k.predict(test_data.drop(['Id', 'Pawpularity'], axis=1))

In [ ]:
p = PCA(random_state=0)
train_data_pca =pd.DataFrame(p.fit_transform(train_data.drop(['Id', 'Pawpularity'], axis=1)))
test_data_pca =pd.DataFrame(p.transform(test_data.drop(['Id', 'Pawpularity'], axis=1)))

In [ ]:
train_data=pd.concat([train_data.reset_index(drop=True),train_data_pca.reset_index(drop=True)],axis=1)
test_data=pd.concat([test_data.reset_index(drop=True),test_data_pca.reset_index(drop=True)],axis=1)

In [ ]:
train, valid = train_test_split(train_data, test_size=0.1)

In [ ]:
input_picture = tf.keras.Input(shape=(128,128,3))
input_meta = tf.keras.Input((32,))
base_model=tf.keras.applications.EfficientNetB1(include_top=True,input_tensor=input_picture,weights="imagenet")
for layer in base_model.layers[:-10]:
    layer.trainable = False
x = tf.keras.layers.Dense(256, activation='relu')(base_model.output)
x=tf.concat([x,input_meta],axis=1)
output=tf.keras.layers.Dense(1,activation="linear")(x)
model = tf.keras.Model(inputs=[input_picture,input_meta], outputs=output)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_squared_error'])

In [ ]:
# Custom data generator for generating batches of images and metadata
class CustomDataGen(tf.keras.utils.Sequence):
  def __init__(self, df, X_col, y_col,
                batch_size,img_size=(128,128),
                shuffle=True, testing_set=False):  
    self.df = df.copy()
    self.df["Id"]="/content/drive/MyDrive/Pawpularity/train/"+self.df["Id"]
    self.X_col = X_col
    self.y_col = y_col
    self.batch_size = batch_size
    self.shuffle = shuffle 
    self.n = len(self.df)
    self.testing_set=testing_set
    self.img_size=img_size
  
  def __get_input(self, path):
    image=tf.keras.utils.load_img(str(path), color_mode='rgb',target_size=self.img_size,interpolation='nearest')
    if self.testing_set:
      image = tf.keras.preprocessing.image.img_to_array(image)/255
    else:
      image=tf.keras.applications.efficientnet.preprocess_input(image)
      image = tf.keras.preprocessing.image.img_to_array(image)/255
      # image = tf.image.random_hue(image, 0.05)
      # image = tf.image.random_contrast(image, 0.95, 1.05)
      # image = tf.image.random_brightness(image, 0.05)
    return image
  
  def __get_data(self, batches):
    # Generates data containing batch_size samples
    path = batches[self.X_col['Id']]
    y_batch= np.asarray(batches[self.y_col['Pawpularity']])
    X_batch = np.asarray([self.__get_input(x) for x in path])
    meta_batch=np.asarray(batches.drop(['Id', 'Pawpularity'], axis=1))
    return tuple([X_batch, meta_batch]),y_batch
  
  def on_epoch_end(self):
    if self.shuffle:
      self.df = self.df.sample(frac=1).reset_index(drop=True)
  
  def __getitem__(self, index):
    batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
    X, y = self.__get_data(batches)        
    return X, y
  
  def __len__(self):
    return self.n // self.batch_size

In [ ]:
#creating the test set i.e. an input tuple of image and metadata of the test set
def get_input(path):
  path="/content/drive/MyDrive/Pawpularity/train/"+path
  image=tf.keras.utils.load_img(str(path), color_mode='rgb',target_size=(128,128),interpolation='nearest')
  image = tf.keras.preprocessing.image.img_to_array(image)/255
  return image
x_test=(test_data.iloc[:,0]).progress_apply(get_input)
x_test=np.stack(x_test).astype(None)
x_test_meta=np.asarray(test_data.drop(['Id', 'Pawpularity'], axis=1))
testset=tuple([x_test, x_test_meta])

In [ ]:
x_train = CustomDataGen(train,
                         X_col={'Id':'Id'},
                         y_col={'Pawpularity':'Pawpularity'},
                         batch_size=100)
x_valid= CustomDataGen(valid,
                         X_col={'Id':'Id'},
                         y_col={'Pawpularity':'Pawpularity'},
                         batch_size=100,testing_set=True)

Normal Fitting

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/pretrainedEfficientNetB1+meta'
model_checkpoint_callback = [tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=2, mode='min', min_delta=0.01)]

In [ ]:
model.fit(x_train,epochs=10,validation_data=x_valid,callbacks=[model_checkpoint_callback])

In [ ]:
y_test=test_data["Pawpularity"]
y_test_pred=model.predict(testset)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))

K fold Cross Validation

In [ ]:
from sklearn.model_selection import KFold
K = 5
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/pretrainedEfficientNetB1+meta'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor="val_root_mean_squared_error",
    mode='min',
    save_best_only=True)
result={}
j=1
for train_index, valid_index in kf.split(train_data):
  # Create data for this fold
  print( "\nFold ", j)
  traingen = CustomDataGen(train_data.iloc[train_index,:],
                         X_col={'Id':'Id'},
                         y_col={'Pawpularity':'Pawpularity'},
                         batch_size=100)
  valgen = CustomDataGen(train_data.iloc[valid_index,:],
                         X_col={'Id':'Id'},
                         y_col={'Pawpularity':'Pawpularity'},
                         batch_size=100,testing_set=True)
  model.fit(traingen,
          validation_data=valgen,
          epochs=5,callbacks=[model_checkpoint_callback])
  j=j+1

In [ ]:
y_test_pred=model.predict(testset)

In [ ]:
y_test=test_data["Pawpularity"]
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))

# Hyparmeter tuning for the best model (Transfer learning with EfficientNetB1)


In [ ]:
target=(128,128)
x_train=datagen.flow_from_dataframe(dataframe=train,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col="Pawpularity",
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode="raw",
                                              batch_size=100)
x_valid=datagen1.flow_from_dataframe(dataframe=valid,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col="Pawpularity",
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode="raw",
                                              batch_size=100)

In [ ]:
x_test=datagen1.flow_from_dataframe(dataframe=test_data,
                                              directory="/content/drive/MyDrive/Pawpularity/train",
                                              x_col="Id",
                                              y_col=None,
                                              weight_col=None,
                                              target_size=target,
                                              color_mode="rgb",
                                              class_mode=None,
                                              batch_size=100)


In [ ]:
target=(128,128)
def model_builder(hp):
  '''
  Args:
    hp - Keras tuner object
  '''
  base_model=tf.keras.applications.EfficientNetB1(include_top=True,weights="imagenet")
  for layer in base_model.layers:
      layer.trainable = False
  model = tf.keras.models.Sequential()
  model.add(base_model)
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(tf.keras.layers.Dense(1,activation="linear"))
  
  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='mean_squared_error', metrics=['mean_squared_error'])
  return model

In [ ]:
tuner = kt.Hyperband(model_builder, # the hypermodel
                     objective='val_mean_squared_error', # objective to optimize
max_epochs=10)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=5)
# Perform hypertuning
tuner.search(x_train,
             epochs=10,
             validation_data=x_valid,callbacks=[stop_early])


In [ ]:
best_hp=tuner.get_best_hyperparameters()[0]

In [ ]:
# Creating the model with the best Hyperparameters
# This model works with the EfficientNetB1 as the pretrained model 
model = tf.keras.models.Sequential([base_model,
  tf.keras.layers.Dense(448, activation='relu'),
  tf.keras.layers.Dense(1,activation="linear")])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_squared_error'])

Normal Fitting

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/checkpointEfficientNetB1besthyperparameters'
model_checkpoint_callback = [tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=2, mode='min', min_delta=0.01)]

In [ ]:
model.fit(x_train,epochs=10,validation_data=x_valid,callbacks=[model_checkpoint_callback])

In [ ]:
y_test=test_data["Pawpularity"]
y_test_pred=model.predict(x_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))

K fold Cross Validation

In [ ]:
#Creating a K fold instance which splits the training set into 5 splits 
from sklearn.model_selection import KFold
K = 5
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Pawpularity/checkpointEfficientNetB1besthyperparameters"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True)
result={}
j=1
for train_index, valid_index in kf.split(train_data):
  # Create data for this fold
  x_train=datagen.flow_from_dataframe(dataframe=train_data.iloc[train_index,:],
                                            directory="/content/drive/MyDrive/Pawpularity/train",
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            weight_col=None,
                                            color_mode="rgb",
                                            target_size=target,
                                            class_mode="raw",
                                            batch_size=100)
  x_valid=datagen1.flow_from_dataframe(dataframe=train_data.iloc[valid_index,:],
                                            directory="/content/drive/MyDrive/Pawpularity/train",
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            weight_col=None,
                                            color_mode="rgb",
                                            class_mode="raw",
                                            target_size=target,
                                            batch_size=100)
  print( "\nFold ", j)
  model.fit(x_train,epochs=5,validation_data=x_valid,callbacks=[model_checkpoint_callback])
  # Accumulate test set predictions
  j+=1
  result[j]=model.predict(x_test)

In [ ]:
y_test=test_data["Pawpularity"]
y_test_pred=model.predict(x_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,y_test_pred))